# Behavioral Cloning

In [1]:
import cv2 as cv
import os
import time
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Simple Network
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D



Using TensorFlow backend.


In [2]:
pathdata = "../data/"

cnames = ['centerIm', 'leftIm','rightIm','steer','throttle','brake','speed']
df = pd.read_csv(pathdata+"driving_log.csv", names=cnames)
# df.head()

# Read center image
xCenter = df['centerIm']
xLeft  = df['leftIm']
xRight = df['rightIm']
ysteer = df['steer']
del df


In [3]:
# Multiple (camera) images as input
def read3camerasImg(pathCenter, pathLeft, pathRight, steering, steerAdj = 0.2):
    """
    input: 
        "pathCenter", "pathLeft" and "pathRight" are all (nx1) vector of strings, containing the path
        of the saved image
        example: pathCenter[0], pathLeft[0] and pathRight[0]
        C:\SelfDrivingCarNanodegree\3BehavioralCloning\data\IMG\center_2018_04_05_13_39_08_155.jpg
        C:\SelfDrivingCarNanodegree\3BehavioralCloning\data\IMG\left_2018_04_05_13_39_08_155.jpg
        C:\SelfDrivingCarNanodegree\3BehavioralCloning\data\IMG\right_2018_04_05_13_39_08_155.jpg
        
        "steering" is a vector of double for the steering angle. Left steering is positive, right is negative
        example: steeing[0] = -0.36
    output:
        X, images in numpy array of (n, h, w, d)
        y, steer value in numpy array of size (n, )
    
    """
    assert len(pathCenter) == len(pathLeft) == len(pathRight) == len(steering)
    images3 = []
    steer = []

    nim = len(xCenter)
    for i in range(nim):
        imgC = cv.cvtColor(cv.imread(xCenter[i]), cv.COLOR_BGR2RGB)
        imgL = cv.cvtColor(cv.imread(xLeft[i]), cv.COLOR_BGR2RGB)
        imgR = cv.cvtColor(cv.imread(xRight[i]), cv.COLOR_BGR2RGB)
        
        # convert 
        # correct steering angle (y) for lect image and right image
        yC = steering[i]
        yL = steering[i] + steerAdj
        yR = steering[i] - steerAdj
        images3.extend([imgC, imgL, imgR])
        steer.extend([yC, yL, yR])
        if (i%1000==0): print("Images imported: " + str(i) + "x 3")
        
    y = np.array(steer)
    X = np.array(images3)
    
    return X, y



In [4]:
def flipBatchImg(images, measurement):
    """
    Create a new Augment batch images with flipped image
    Input:  images:          (n, w, d, 3)
            measurement:     (n, )
    """
    # data augmentation: Flipped image
    
    assert images.shape[0] == measurement.shape[0]
    
    fImages = []
    fMeasurement = []
    
    for i, img in enumerate(images):
        image_flipped = np.fliplr(img)
        measurement_flipped = - measurement[i]
        fImages.append(image_flipped)
        fMeasurement.append(measurement_flipped)
        if (i%1000==0): print("Images flipped: " + str(i))
        
    newImages = np.array(fImages)
    newMeasurement = np.array(fMeasurement)
        
    return newImages, newMeasurement


In [ ]:
# # impath = xCenter[0]
# # img = cv.imread(impath)
# images = []
# for i, ipath in enumerate(xCenter):
#     img = cv.imread(ipath)
#     images.append(img)
# #     print(i, img)

# X = np.array(images)
# y = np.array(y)
# # plt.imshow(img)
# print(X.shape)
# print(y.shape)

# timg = cv.imread(xCenter[100])
# ## red and blue colors are inverted
# timg2 = cv.cvtColor(timg, cv.COLOR_RGB2BGR)
    
# timg3 = cv.cvtColor(timg, cv.COLOR_BGR2RGB)

# timg4 =  cv.cvtColor(cv.imread(xCenter[3]), cv.COLOR_BGR2RGB)
# plt.imshow(timg4)



In [ ]:
# nim = len(xCenter)
# t1 = []
# t2 = []
# for i in range(nim):
#     yC = ysteer[i]
#     yL = ysteer[i] + 0.2
#     yR = ysteer[i] - 0.2
#     t1.extend([yC, yL, yR])
#     t2.append([yC, yL, yR])

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=0)
# print(X_train.shape)
# print(X_val.shape)
# print(y_train.shape)

print(np.array(t2))

In [5]:
X, y = read3camerasImg(xCenter, xLeft, xRight, steering=ysteer, steerAdj=0.2)

Images imported: 0x 3
Images imported: 1000x 3
Images imported: 2000x 3
Images imported: 3000x 3
Images imported: 4000x 3
Images imported: 5000x 3
Images imported: 6000x 3
Images imported: 7000x 3
Images imported: 8000x 3
Images imported: 9000x 3


In [6]:
# if SAVE:
#         save_path = saver.save(sess, MODEL_PATH)
#         print('Trained model saved at: %s' % save_path)
#         # Save accuracy history
#         print('Accuracy history saved at accuracy_history.p')
#         with open('accuracy_history.p', 'wb') as f:
#             pickle.dump(accuracy_history, f)
    

In [7]:
from random import sample

n = X.shape[0]

#sample without replacement. add 1/4 sample where each image is flipped
select = sample(range(n), int(n/4))



In [8]:
print(X.shape)
print(y.shape)

# Save example images in example directory
expath = "example/"
if not os.path.isdir(expath): os.makedirs(expath)

# Note: cv has default BGR. As I read with BGR and write with BGR, no need of color conversion
imgC = cv.imread(xCenter[123])
imgL = cv.imread(xLeft[123])
imgR = cv.imread(xRight[123])
cv.imwrite(expath + "cameraLeft.png", imgL)
cv.imwrite(expath + "cameraRight.png", imgR)
cv.imwrite(expath + "cameraCenter.png", imgC)


(27300, 160, 320, 3)
(27300,)


True

In [9]:
x1, y1 = flipBatchImg(X[select,], y[select,])
X = np.concatenate((X,x1))
y = np.concatenate((y,y1))

Images flipped: 0
Images flipped: 1000
Images flipped: 2000
Images flipped: 3000
Images flipped: 4000
Images flipped: 5000
Images flipped: 6000


In [10]:
print(X.shape)
print(y.shape)

(34125, 160, 320, 3)
(34125,)


In [11]:

model = Sequential()

# Data preprocessing. 
## normalize. Improve substantially
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape = (160,320,3)) )

## Cropping. Remove top 70 pixels and bottom 25 pixels
model.add(Cropping2D(cropping = ((70,25), (0,0))))

# MyNet architecture
model.add(Conv2D(9, (10,10), padding='valid', activation = "relu"))
model.add(MaxPooling2D())

# 5x5 convolution, 6 filters
model.add(Conv2D(27, (6,6), padding='valid', activation = None))
model.add(Conv2D(81, (3,3), padding='valid', activation = "relu"))
model.add(MaxPooling2D())

# Flat layer 
model.add(Flatten())

# relu(xw +b)
model.add(Dense(150, activation = "relu"))
model.add(Dense(50, activation = "relu"))
model.add(Dense(1))

model.compile(loss="mse", optimizer="adam")

model.fit(X, y, validation_split=0.2, shuffle=True, epochs = 5)
model.save("model1.h5")


Train on 27300 samples, validate on 6825 samples
Epoch 1/5
27300/27300 [==============================] - 3729s 137ms/step - loss: 0.0366 - val_loss: 0.0478
Epoch 2/5
27300/27300 [==============================] - 3692s 135ms/step - loss: 0.0281 - val_loss: 0.0384
Epoch 3/5
27300/27300 [==============================] - 3732s 137ms/step - loss: 0.0253 - val_loss: 0.0396
Epoch 4/5
27300/27300 [==============================] - 3707s 136ms/step - loss: 0.0233 - val_loss: 0.0388
Epoch 5/5
27300/27300 [==============================] - 3710s 136ms/step - loss: 0.0204 - val_loss: 0.0414


In [14]:
model.summary()
# # reload model
# from keras.models import load_model
# model1 = load_model("model1.h5")

# model1.fit(X, y, validation_split=0.2, shuffle=True, epochs = 5)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 311, 9)        2709      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 155, 9)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 150, 27)       8775      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 148, 81)       19764     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 74, 81)        0         
__________

In [13]:
# Visualize Loss
from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit(X, y, validation_split=0.2, shuffle=True, epochs = 5, verbose=1)
# history_object = model.fit_generator(train_generator, samples_per_epoch =len(train_samples), 
#                                      validation_data = validation_generator, 
#                                      nb_val_samples = len(validation_samples), nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('Model Mean Squared Error (MSE) loss')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['Training set', 'Validation set'], loc='upper right')
plt.show()

Train on 27300 samples, validate on 6825 samples
Epoch 1/5
  544/27300 [..............................] - ETA: 59:40 - loss: 0.0169

KeyboardInterrupt: 